# Model derivation

This classifier models:

$P(y|x) \propto P(x|y)P(y)$

where

- $P(x|y)$ is estimated non-parametrically using KDE, one KDE per class.

- $P(y)$ is the class prior, estimated from class frequencies.

Let's step through this code and discuss the essential features.

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KernelDensity

class bayesian_kde_classifier(BaseEstimator, ClassifierMixin):
	"""Bayesian generative classification based on KDE
	Parameters
	----------
	bandwidth : float
	the kernel bandwidth within each class
	kernel : str
	the kernel name, passed to KernelDensity
	"""

	def __init__(self, bandwidth=1.0, kernel='gaussian'):
		self.bandwidth = bandwidth
		self.kernel = kernel

	def fit(self, X, y):
		self.classes_ = np.sort(np.unique(y))
		training_sets = [X[y == yi] for yi in self.classes_]
		self.models_ = [KernelDensity(bandwidth=self.bandwidth,
						kernel=self.kernel).fit(Xi)
				for Xi in training_sets]
		self.logpriors_ = [np.log(Xi.shape[0] / X.shape[0])
						for Xi in training_sets]
		return self

	def predict_proba(self, X):
		logprobs = np.array([model.score_samples(X)
				for model in self.models_]).T
		result = np.exp(logprobs + self.logpriors_)
		return result / result.sum(1, keepdims=True)

	def predict(self, X):
		return self.classes_[np.argmax(self.predict_proba(X), 1)]

The general approach for Bayesian generative classification is this:

1. Split the training data by label.

2. For each set, fit a KDE to obtain a generative model of the data. This allows you for any observation $x$ and label $y$ to compute a likelihood $P(x|y)$

3. From the number of examples of each class in the training set, compute the class prior, $P(y)$.


4. For an unknown point $x$, the posterior probability for each class is $P(y|x) \propto P(x|y)P(y)$. The class which maximizes this posterior is the label assigned to the point.

## Anatomy of the custom estimator


 1. Each estimator in Scikit-Learn is a class, and it is most convenient for this class to inherit from the BaseEstimator class as well as the appropriate mixin, which provides standard functionality. For example, among other things, here the BaseEstimator contains the logic necessary to clone/copy an estimator for use in a cross-validation procedure, and ClassifierMixin defines a default score() method used by such routines. We also provide a doc string, which will be captured by IPython's help functionality.

In [ ]:
# 1

from sklearn.base import BaseEstimator, ClassifierMixin

class KDEClassifier(BaseEstimator, ClassifierMixin):
    """Bayesian generative classification based on KDE
    
    Parameters
    ----------
    bandwidth : float
        the kernel bandwidth within each class
    kernel : str
        the kernel name, passed to KernelDensity
    """

2. This is the actual code that is executed when the object is instantiated with KDEClassifier(). In Scikit-Learn, it is important that initialization contains no operations other than assigning the passed values by name to self. This is due to the logic contained in BaseEstimator required for cloning and modifying estimators for cross-validation, grid search, and other functions. Similarly, all arguments to __ init __ should be explicit: i.e. *args or **kwargs should be avoided, as they will not be correctly handled within cross-validation routines.

In [ ]:
# 2

    def __init__(self, bandwidth=1.0, kernel='gaussian'):
        self.bandwidth = bandwidth
        self.kernel = kernel

### 3.

Here we find the unique classes in the training data, train a KernelDensity model for each class, and compute the class priors based on the number of input samples. Finally, fit() should always return self so that we can chain commands. For example:

`label = model.fit(X, y).predict(X)`


Why logs?

KDE outputs log-likelihoods, adding logs avoids underflow:

$logP(x,y)=logP(x|y)+logP(y)$

In [ ]:
# 3    

    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y)) # Extracts unique class labels from y
        training_sets = [X[y == yi] for yi in self.classes_] # Splits X into subsets for each class
        self.models_ = [KernelDensity(bandwidth=self.bandwidth, # Initializes a KDE model for each class,
                                      kernel=self.kernel).fit(Xi) # Fits the model to the class-specific data, this is P(x|y).
                        for Xi in training_sets]
        self.logpriors_ = [np.log(Xi.shape[0] / X.shape[0]) # Computes the log prior probabilities for each class, this is P(y).
                           for Xi in training_sets]
        return self

### 4.

Purpose: Compute posterior class probabilities $P(y|x)$. Because this is a probabilistic classifier, we first implement predict_proba() which returns an array of class probabilities of shape `[n_samples, n_classes]`. Entry `[i, j]` of this array is the posterior probability that sample i is a member of class j.

This is Bayes’ rule in vectorized form

$P(y|x)=\frac{P(x|y)P(y)}{\sum_{c'} P(x|c')P(c')}$

In [ ]:
# 4

def predict_proba(self, X):
		logprobs = np.array([model.score_samples(X) # Computes log P(x|y)
				for model in self.models_]).T # Transposes to shape (n_samples, n_classes)
		result = np.exp(logprobs + self.logpriors_) # Computes P(x|y) * P(y)
		return result / result.sum(1, keepdims=True) # Normalizes across classes to get P(y|x)

### 5.

Finally, the `predict()` method uses these probabilities and simply returns the class with the largest probability.

$\hat{y}=arg \max_{y}P(y|x)$

In [ ]:
# 5

def predict(self, X):
		return self.classes_[np.argmax(self.predict_proba(X), 1)] # Predicts the class with the highest posterior probability